Make an ETL pipeline that loads from your `tickets.json` file

In [1]:
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List
import json


# **** SETUP ****

# global variables for file system/ loading JSON file
JSON_DATA = "../data/tickets.json"
# project paths  
PROJECT_NAME = "abstract-flame-407818"
DATASET_NAME = "air_travel"


#schemas we will use for each table 

TABLE_METADATA = {
    'airlines': {
        'table_name': 'airlines',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('airline_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_icao', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_callsign', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_country', 'string', mode='REQUIRED'),
        ],
    },
    'airports': {
        'table_name': 'airports',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('airport_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airport_city', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airport_country', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airport_iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airport_icao', 'string', mode='REQUIRED'),
        ],
    },
    'passengers': {
        'table_name': 'passengers',
        'schema': [
            # schema for passengers
            bigquery.SchemaField('pass_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_gender', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_birth_date', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_email', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_street', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_city', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_state', 'string', mode='REQUIRED'),
            bigquery.SchemaField('pass_zip', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('start_date', 'string', mode='REQUIRED'),
            bigquery.SchemaField('end_date', 'string', mode='NULLABLE'),
            bigquery.SchemaField('UUID', 'string', mode='REQUIRED'),
        ],
    },
    'tickets': {
        'table_name': 'tickets',
        'schema': [
            # schema for tickets
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED'),
            bigquery.SchemaField('confirmation', 'string', mode='REQUIRED'),
            bigquery.SchemaField('origin_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('dest_name', 'string', mode='NULLABLE'),
            bigquery.SchemaField('ticket_date', 'string', mode='REQUIRED'),
            bigquery.SchemaField('price', 'float', mode='REQUIRED'),
            bigquery.SchemaField('seat', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('origin_iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('UUID', 'string', mode='REQUIRED'),
        ],
    },
}

# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level

Create dataset in bigquery

In [5]:
#establish dataset ID
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
client = bigquery.Client()

def create_dataset(client: bigquery.Client ,dataset_id: str, location: str = 'US') -> None:
    """create a bigquery dataset"""
    # create the dataset
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location
    dataset = client.create_dataset(dataset, exists_ok=True)
    logger.info(f"Created Air Travel dataset: {dataset.full_dataset_id}")

[DEBUG][2023-12-23 13:27:21,915][_default:0255] : Checking /Users/kairo/.creds/dsa-deb-sa.json for explicit credentials as part of auth process...
[DEBUG][2023-12-23 13:27:21,917][_default:0255] : Checking /Users/kairo/.creds/dsa-deb-sa.json for explicit credentials as part of auth process...


Load tickets JSON into pandas dataframe

In [12]:
# Load JSON into pandas dataframe
df = pd.read_json('./data/tickets.json', lines=True)
def extract_columns(dataframe, column_name): 
        df_columns = dataframe[column_name].apply(pd.Series)
        df_columns = df_columns.rename(columns={col: column_name + '_' + col for col in df_columns.columns})
        extract_df = pd.concat([dataframe.drop([column_name], axis=1), df_columns], axis=1)
        return extract_df

airline_df = extract_columns(df, 'airline')
#df2 = extract_columns(df1, 'origin')
#df3 = extract_columns(df2, 'destination')
#final_df = extract_columns(df3, 'passenger')
display(airline_df)

,eticket_num,confirmation,ticket_date,price,seat,status,origin,destination,passenger,airline_name,airline_iata,airline_icao,airline_callsign,airline_country
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,{'name': 'Montreal / Pierre Elliott Trudeau In...,{'name': 'Chicago Midway International Airport...,"{'first_name': 'Robert', 'last_name': 'Brown',...",China Eastern Airlines,MU,CES,CHINA EASTERN,China
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,"{'name': 'Longdongbao Airport', 'city': 'Guiya...","{'name': 'Ninoy Aquino International Airport',...","{'first_name': 'Laura', 'last_name': 'Kent', '...",Hawaiian Airlines,HA,HAL,HAWAIIAN,United States
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,{'name': 'Licenciado Gustavo Díaz Ordaz Intern...,"{'name': 'Ibiza Airport', 'city': 'Ibiza', 'co...","{'first_name': 'Lisa', 'last_name': 'Tucker', ...",Wizz Air,W6,WZZ,WIZZ AIR,Hungary
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,"{'name': 'El Tepual Airport', 'city': 'Puerto ...","{'name': 'Gdańsk Lech Wałęsa Airport', 'city':...","{'first_name': 'Matthew', 'last_name': 'Yates'...",AirAsia,AK,AXM,ASIAN EXPRESS,Malaysia
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,{'name': 'Baltimore/Washington International T...,"{'name': 'London Gatwick Airport', 'city': 'Lo...","{'first_name': 'Megan', 'last_name': 'Villanue...",Xiamen Airlines,MF,CXA,XIAMEN AIR,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,030-327889-3270,MVIBWK,2022-03-24,600.37,23E,active,"{'name': 'Don Mueang International Airport', '...",None,"{'first_name': 'Janice', 'last_name': 'Zamora'...",Hainan Airlines,HU,CHH,HAINAN,China
4092,513-551750-0628,WZZGGB,2022-03-21,583.41,24F,active,{'name': 'Governador Aluízio Alves Internation...,{'name': 'Charles de Gaulle International Airp...,"{'first_name': 'Lisa', 'last_name': 'Tucker', ...",Malaysia Airlines,MH,MAS,MALAYSIAN,Malaysia
4093,118-106280-2530,WUD4KR,2022-03-22,203.45,17H,active,"{'name': 'Taiyuan Wusu Airport', 'city': 'Taiy...","{'name': 'Miami International Airport', 'city'...","{'first_name': 'Janice', 'last_name': 'Zamora'...",Frontier Airlines,F9,FFT,FRONTIER FLIGHT,United States
4094,961-278558-3018,VI5039,2022-03-21,554.59,18G,active,"{'name': 'San Carlos De Bariloche Airport', 'c...","{'name': 'Hamad International Airport', 'city'...","{'first_name': 'Corey', 'last_name': 'Cook', '...",Royal Air Maroc,AT,RAM,ROYALAIR MAROC,Morocco


Load airports dimension table 

In [21]:
def airlines_get_unique(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a unique set of products from the tickets dataframe 

    Args:
        df (pd.DataFrame): tickets dataframe

    Returns:
        pd.DataFrame: return value. unique airline names 
    """
    logger.debug(f"getting unique names...")
    # group by unique columns and only select them
    cols = ['airline_name', 'airline_iata', 'airline_icao', 'airline_callsign', 'airline_country']
    df = df.groupby(cols).all()
    df = df.reset_index().loc[:, cols]
    return df
airline_df = airlines_get_unique(airline_df)
display(airline_df)


[DEBUG][2023-12-23 14:00:14,279][779889110:0011] : getting unique iatas...


,airline_name,airline_iata,airline_icao,airline_callsign,airline_country
0,Air Canada,AC,ACA,AIR CANADA,Canada
1,Air China,CA,CCA,AIR CHINA,China
2,Air France,AF,AFR,AIRFRANS,France
3,Air New Zealand,NZ,ANZ,NEW ZEALAND,New Zealand
4,AirAsia,AK,AXM,ASIAN EXPRESS,Malaysia
5,Alaska Airlines,AS,ASA,Inc.,ALASKA
6,Allegiant Air,G4,AAY,ALLEGIANT,United States
7,American Airlines,AA,AAL,AMERICAN,United States
8,British Airways,BA,BAW,SPEEDBIRD,United Kingdom
9,Cape Air,9K,KAP,CAIR,United States


Load the airlines dataframe columns into a bigquery table

In [22]:
airline_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['airlines']['table_name']}"
airline_schema = schema=TABLE_METADATA['airlines']['schema']

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")
load_table(airline_df, client, airline_table_name, airline_schema)

[INFO ][2023-12-23 14:00:28,397][4006487939:0032] : loading table: 'abstract-flame-407818.air_travel.airlines'
[DEBUG][2023-12-23 14:00:28,403][connectionpool:0292] : Resetting dropped connection: bigquery.googleapis.com
[DEBUG][2023-12-23 14:00:29,472][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/abstract-flame-407818/jobs?uploadType=multipart HTTP/1.1" 200 1918
[DEBUG][2023-12-23 14:00:29,654][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/2eab7de6-d2ce-4707-a1cf-86bc0daeeec5?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 14:00:29,655][retry:0233] : Retrying due to , sleeping 0.3s ...
[DEBUG][2023-12-23 14:00:30,071][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/2eab7de6-d2ce-4707-a1cf-86bc0daeeec5?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 14:00:30,072][retry

Load airports columns from JSON file  

In [52]:
airport_df = extract_columns(df, 'origin')
airport_df = airport_df.rename(columns={'origin_name': 'airport_name', 'origin_city': 'airport_city', 'origin_country': 'airport_country', 'origin_iata': 'airport_iata', 'origin_icao': 'airport_icao'})
display(airport_df)

,eticket_num,confirmation,ticket_date,price,seat,status,airline,destination,passenger,airport_name,airport_city,airport_country,airport_iata,airport_icao,origin_latitude,origin_longitude,origin_altitude,origin_tz_timezone
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,"{'name': 'China Eastern Airlines', 'iata': 'MU...",{'name': 'Chicago Midway International Airport...,"{'first_name': 'Robert', 'last_name': 'Brown',...",Montreal / Pierre Elliott Trudeau Internationa...,Montreal,Canada,YUL,CYUL,45.47,-73.74,118.0,America/Toronto
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,"{'name': 'Hawaiian Airlines', 'iata': 'HA', 'i...","{'name': 'Ninoy Aquino International Airport',...","{'first_name': 'Laura', 'last_name': 'Kent', '...",Longdongbao Airport,Guiyang,China,KWE,ZUGY,26.54,106.80,3736.0,Asia/Shanghai
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,"{'name': 'Wizz Air', 'iata': 'W6', 'icao': 'WZ...","{'name': 'Ibiza Airport', 'city': 'Ibiza', 'co...","{'first_name': 'Lisa', 'last_name': 'Tucker', ...",Licenciado Gustavo Díaz Ordaz International Ai...,Puerto Vallarta,Mexico,PVR,MMPR,20.68,-105.25,23.0,America/Mexico_City
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,"{'name': 'AirAsia', 'iata': 'AK', 'icao': 'AXM...","{'name': 'Gdańsk Lech Wałęsa Airport', 'city':...","{'first_name': 'Matthew', 'last_name': 'Yates'...",El Tepual Airport,Puerto Montt,Chile,PMC,SCTE,-41.44,-73.09,294.0,America/Santiago
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,"{'name': 'Xiamen Airlines', 'iata': 'MF', 'ica...","{'name': 'London Gatwick Airport', 'city': 'Lo...","{'first_name': 'Megan', 'last_name': 'Villanue...",Baltimore/Washington International Thurgood Ma...,Baltimore,United States,BWI,KBWI,39.18,-76.67,146.0,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,030-327889-3270,MVIBWK,2022-03-24,600.37,23E,active,"{'name': 'Hainan Airlines', 'iata': 'HU', 'ica...",None,"{'first_name': 'Janice', 'last_name': 'Zamora'...",Don Mueang International Airport,Bangkok,Thailand,DMK,VTBD,13.91,100.61,9.0,Asia/Bangkok
4092,513-551750-0628,WZZGGB,2022-03-21,583.41,24F,active,"{'name': 'Malaysia Airlines', 'iata': 'MH', 'i...",{'name': 'Charles de Gaulle International Airp...,"{'first_name': 'Lisa', 'last_name': 'Tucker', ...",Governador Aluízio Alves International Airport,Natal,Brazil,NAT,SBSG,-5.77,-35.38,272.0,America/Fortaleza
4093,118-106280-2530,WUD4KR,2022-03-22,203.45,17H,active,"{'name': 'Frontier Airlines', 'iata': 'F9', 'i...","{'name': 'Miami International Airport', 'city'...","{'first_name': 'Janice', 'last_name': 'Zamora'...",Taiyuan Wusu Airport,Taiyuan,China,TYN,ZBYN,37.75,112.63,2575.0,Asia/Shanghai
4094,961-278558-3018,VI5039,2022-03-21,554.59,18G,active,"{'name': 'Royal Air Maroc', 'iata': 'AT', 'ica...","{'name': 'Hamad International Airport', 'city'...","{'first_name': 'Corey', 'last_name': 'Cook', '...",San Carlos De Bariloche Airport,San Carlos De Bariloche,Argentina,BRC,SAZS,-41.15,-71.16,2774.0,America/Argentina/Salta


Find unique origin iatas for airports table

In [70]:
def airports_get_unique(df: pd.DataFrame) -> pd.DataFrame:

    logger.debug(f"getting unique names...")
    df = df[['airport_name', 'airport_city', 'airport_country', 'airport_iata', 'airport_icao']]
    df = df.drop_duplicates(subset='airport_iata', keep='first')
    return df 

airport_df = airports_get_unique(airport_df)
airport_df = airport_df.dropna(subset='airport_name')
airport_df = airport_df.reset_index()
airport_df = airport_df.drop(columns='index')
display(airport_df)

[DEBUG][2023-12-23 16:32:32,259][1318332233:0003] : getting unique names...


,airport_name,airport_city,airport_country,airport_iata,airport_icao
0,Montreal / Pierre Elliott Trudeau Internationa...,Montreal,Canada,YUL,CYUL
1,Longdongbao Airport,Guiyang,China,KWE,ZUGY
2,Licenciado Gustavo Díaz Ordaz International Ai...,Puerto Vallarta,Mexico,PVR,MMPR
3,El Tepual Airport,Puerto Montt,Chile,PMC,SCTE
4,Baltimore/Washington International Thurgood Ma...,Baltimore,United States,BWI,KBWI
...,...,...,...,...,...
386,Mactan Cebu International Airport,Cebu,Philippines,CEB,RPVM
387,Mariscal Sucre International Airport,Quito,Ecuador,UIO,SEQM
388,Capitan Oriel Lea Plaza Airport,Tarija,Bolivia,TJA,SLTJ
389,Palm Beach International Airport,West Palm Beach,United States,PBI,KPBI


Load airports dataframe into a bigquery table 

In [71]:
airport_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['airports']['table_name']}"
airport_schema = schema=TABLE_METADATA['airports']['schema']

load_table(airport_df, client, airport_table_name, airport_schema)

[INFO ][2023-12-23 16:32:37,978][4006487939:0032] : loading table: 'abstract-flame-407818.air_travel.airports'
[DEBUG][2023-12-23 16:32:38,943][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/abstract-flame-407818/jobs?uploadType=multipart HTTP/1.1" 200 1914
[DEBUG][2023-12-23 16:32:39,120][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/a3b79586-ad74-40da-bf30-c2321760d864?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 16:32:39,121][retry:0233] : Retrying due to , sleeping 0.3s ...
[DEBUG][2023-12-23 16:32:39,580][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/a3b79586-ad74-40da-bf30-c2321760d864?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 16:32:39,581][retry:0233] : Retrying due to , sleeping 0.9s ...
[DEBUG][2023-12-23 16:32:40,652][connectionpool:0546] : https://b

Load passengers columns into dataframe 

In [136]:
passenger_df = extract_columns(df, 'passenger')
display(passenger_df.head())

,eticket_num,confirmation,ticket_date,price,seat,status,airline,origin,destination,passenger_first_name,passenger_last_name,passenger_gender,passenger_birth_date,passenger_email,passenger_street,passenger_city,passenger_state,passenger_zip
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,"{'name': 'China Eastern Airlines', 'iata': 'MU...",{'name': 'Montreal / Pierre Elliott Trudeau In...,{'name': 'Chicago Midway International Airport...,Robert,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,"{'name': 'Hawaiian Airlines', 'iata': 'HA', 'i...","{'name': 'Longdongbao Airport', 'city': 'Guiya...","{'name': 'Ninoy Aquino International Airport',...",Laura,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,"{'name': 'Wizz Air', 'iata': 'W6', 'icao': 'WZ...",{'name': 'Licenciado Gustavo Díaz Ordaz Intern...,"{'name': 'Ibiza Airport', 'city': 'Ibiza', 'co...",Lisa,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,"{'name': 'AirAsia', 'iata': 'AK', 'icao': 'AXM...","{'name': 'El Tepual Airport', 'city': 'Puerto ...","{'name': 'Gdańsk Lech Wałęsa Airport', 'city':...",Matthew,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,"{'name': 'Xiamen Airlines', 'iata': 'MF', 'ica...",{'name': 'Baltimore/Washington International T...,"{'name': 'London Gatwick Airport', 'city': 'Lo...",Megan,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177


Concatenate first and last names of passengers and transform columns

In [137]:
def passenger_transform_and_unique(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(columns= 'pass_name', errors='ignore')
    df['pass_name'] = df['passenger_first_name'] + ' ' + df['passenger_last_name']
    df = df.rename(columns={'passenger_gender': 'pass_gender',
                            'passenger_birth_date': 'pass_birth_date',
                            'passenger_email': 'pass_email',
                            'passenger_street': 'pass_street',
                            'passenger_city': 'pass_city',
                            'passenger_state': 'pass_state',
                            'passenger_zip': 'pass_zip',
                            'ticket_date': 'start_date'})
    df['end_date'] = None
    df = df[['pass_name', 'pass_gender', 'pass_birth_date', 'pass_email', 'pass_street', 'pass_city', 'pass_state', 'pass_zip', 'start_date', 'end_date']]
    df = df.drop_duplicates(subset='pass_name')
    df = df.reset_index()
    df = df.drop(columns='index')
    df = df.astype({'pass_zip': 'int64'})
    return df
passenger_df = passenger_transform_and_unique(passenger_df)
display(passenger_df)



,pass_name,pass_gender,pass_birth_date,pass_email,pass_street,pass_city,pass_state,pass_zip,start_date,end_date
0,Robert Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,2022-03-23,None
1,Laura Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,2022-03-23,None
2,Lisa Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,1093,2022-03-21,None
3,Matthew Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,2022-03-22,None
4,Megan Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,2022-03-24,None
5,Sarah Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,2022-03-23,None
6,Seth Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,2022-03-23,None
7,Jennifer Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,2022-03-22,None
8,Becky Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,2022-03-23,None
9,Ronald Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,2022-03-23,None


Generate UUIDs for each passenger

In [138]:
import uuid

def create_uuid(df):
    for _ in range(len(df.index)):
        df['UUID'] = df.apply(lambda _: uuid.uuid4(), axis=1)
        df = df.astype({'UUID': 'string'})
    return df

passenger_df = create_uuid(passenger_df)
display(passenger_df)

,pass_name,pass_gender,pass_birth_date,pass_email,pass_street,pass_city,pass_state,pass_zip,start_date,end_date,UUID
0,Robert Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,2022-03-23,None,bb7fa7ea-cc63-426f-b581-9d95b83f34b9
1,Laura Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,2022-03-23,None,5883b402-a7c0-4015-87b9-4def1cf2f277
2,Lisa Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,1093,2022-03-21,None,fced00a7-9db5-473c-a3a8-9a6f6990c017
3,Matthew Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,2022-03-22,None,447196f3-1b0d-48e3-a8dd-3febf0308408
4,Megan Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,2022-03-24,None,a4df49c5-fd8c-49cb-8b57-0feb2e5bbaa2
5,Sarah Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,2022-03-23,None,681483b7-33a3-44d8-a699-ec3d346a1fc5
6,Seth Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,2022-03-23,None,ca88034e-94d9-4490-9cc1-6e12a3e12e11
7,Jennifer Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,2022-03-22,None,054d3470-50fb-4b56-98e9-c8c57425e4e4
8,Becky Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,2022-03-23,None,e31a9386-9204-47d3-8ba8-8a518068db98
9,Ronald Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,2022-03-23,None,eac0646b-d1f4-4883-b577-9e641330a7f7


load passengers columns into bigquery table

In [139]:
passenger_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['passengers']['table_name']}"
passenger_schema = schema=TABLE_METADATA['passengers']['schema']
load_table(passenger_df, client, passenger_table_name, passenger_schema)

[INFO ][2023-12-23 18:02:56,040][4006487939:0032] : loading table: 'abstract-flame-407818.air_travel.passengers'
[DEBUG][2023-12-23 18:02:56,786][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/abstract-flame-407818/jobs?uploadType=multipart HTTP/1.1" 200 2619
[DEBUG][2023-12-23 18:02:56,951][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/96f5d1a4-be22-4186-8a90-17deb6892fd9?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 18:02:56,952][retry:0233] : Retrying due to , sleeping 0.6s ...
[DEBUG][2023-12-23 18:02:57,662][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/96f5d1a4-be22-4186-8a90-17deb6892fd9?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 18:02:57,664][retry:0233] : Retrying due to , sleeping 0.5s ...
[DEBUG][2023-12-23 18:02:58,318][connectionpool:0546] : https:/

Load tickets columns from ticket JSON into a pandas dataframe 

In [204]:
ticket_df = extract_columns(df, 'passenger')
ticket_df = extract_columns(ticket_df, 'origin')
ticket_df = extract_columns(ticket_df, 'airline')
ticket_df = extract_columns(ticket_df, 'destination')
#ticket_df = ticket_df.rename(columns={'destination_name': 'dest_name'})
#ticket_df = ticket_df[['eticket_num', 'confirmation', 'origin_name', 'dest_name', 'ticket_date', 'price', 'seat', 'origin_iata', 'airline_iata', 'passenger_email']]
#ticket_df = ticket_df.dropna(subset='origin_name')
display(ticket_df)

,eticket_num,confirmation,ticket_date,price,seat,status,passenger_first_name,passenger_last_name,passenger_gender,passenger_birth_date,...,airline_country,destination_name,destination_city,destination_country,destination_iata,destination_icao,destination_latitude,destination_longitude,destination_altitude,destination_tz_timezone
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,Robert,Brown,M,1969-02-17,...,China,Chicago Midway International Airport,Chicago,United States,MDW,KMDW,41.79,-87.75,620.0,America/Chicago
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Laura,Kent,F,1998-08-05,...,United States,Ninoy Aquino International Airport,Manila,Philippines,MNL,RPLL,14.51,121.02,75.0,Asia/Manila
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Lisa,Tucker,F,1965-01-22,...,Hungary,Ibiza Airport,Ibiza,Spain,IBZ,LEIB,38.87,1.37,24.0,Europe/Madrid
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,Matthew,Yates,NB,1975-03-31,...,Malaysia,Gdańsk Lech Wałęsa Airport,Gdansk,Poland,GDN,EPGD,54.38,18.47,489.0,Europe/Warsaw
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Megan,Villanueva,NB,1945-08-14,...,China,London Gatwick Airport,London,United Kingdom,LGW,EGKK,51.15,-0.19,202.0,Europe/London
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,030-327889-3270,MVIBWK,2022-03-24,600.37,23E,active,Janice,Zamora,F,2000-06-14,...,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4092,513-551750-0628,WZZGGB,2022-03-21,583.41,24F,active,Lisa,Tucker,F,1965-01-22,...,Malaysia,Charles de Gaulle International Airport,Paris,France,CDG,LFPG,49.01,2.55,392.0,Europe/Paris
4093,118-106280-2530,WUD4KR,2022-03-22,203.45,17H,active,Janice,Zamora,F,2000-06-14,...,United States,Miami International Airport,Miami,United States,MIA,KMIA,25.79,-80.29,8.0,America/New_York
4094,961-278558-3018,VI5039,2022-03-21,554.59,18G,active,Corey,Cook,M,1983-06-14,...,Morocco,Hamad International Airport,Doha,Qatar,DOH,OTHH,25.27,51.61,13.0,None


slight transformations to the tickets dataframe

In [206]:
def transform_ticket(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={'destination_name': 'dest_name', 'passenger_email': 'pass_email'})
    df = df[['eticket_num', 'confirmation', 'origin_name', 'dest_name', 'ticket_date', 'price', 'seat', 'origin_iata', 'airline_iata', 'pass_email']]
    df = df.dropna(subset='origin_name')
    df = df.reset_index()
    df = df.drop(columns='index')
    return df
ticket_df = transform_ticket(ticket_df)
display(ticket_df.head())

,eticket_num,confirmation,origin_name,dest_name,ticket_date,price,seat,origin_iata,airline_iata,pass_email
0,498-938211-0795,ZVFDC4,Montreal / Pierre Elliott Trudeau Internationa...,Chicago Midway International Airport,2022-03-23,723.42,31I,YUL,MU,robert.brown.69@hotmail.com
1,482-850738-6048,IL5GUI,Longdongbao Airport,Ninoy Aquino International Airport,2022-03-23,765.18,29B,KWE,HA,laura.kent.98@hotmail.com
2,275-207321-8092,CYEFBC,Licenciado Gustavo Díaz Ordaz International Ai...,Ibiza Airport,2022-03-21,753.89,26I,PVR,W6,lisa.tucker.65@hotmail.com
3,246-793315-3102,ZNGPC2,El Tepual Airport,Gdańsk Lech Wałęsa Airport,2022-03-22,793.89,15A,PMC,AK,matthew.yates.75@yahoo.com
4,091-128904-1226,MGSBD9,Baltimore/Washington International Thurgood Ma...,London Gatwick Airport,2022-03-24,820.25,17F,BWI,MF,megan.villanueva.45@hotmail.com


Lookup UUIDs from passengers table and incorporate them into the tickets dataframe

In [214]:
def lookup_uuid(df: pd.DataFrame, client: bigquery.Client, passenger_table: str = 'passengers') -> pd.DataFrame:
    """
    Lookup UUID from passengers table.

    Args:
        df (pd.DataFrame): receipts dataframe
        client (bigquery.Client): bigquery client
        customer_table (str, optional): passengers table name. Defaults to 'customers'.

    Returns:
        pd.DataFrame: return dataframe
    """
    # passengers table query
    query = f"""
    SELECT
    UUID,
    pass_email
    FROM 
    `{PROJECT_NAME}.{DATASET_NAME}.passengers`
    """
    # query bigquery table into dataframe
    tdf = client.query(query).to_dataframe()
    # index by product_name for the join
    tdf = tdf.set_index(keys='pass_email')
    logger.info(f"query {len(tdf.index)} rows from passengers table")

    # drop UUID column if it already exists
    df = df.drop(columns=['UUID'], errors='ignore')

    logger.debug(f"preparing for join")

    # join with the main dataframe to get UUID
    df = df.join(tdf, on='pass_email', how='inner')
    logger.info(f"looked up UUID")
    # reset index
    df = df.reset_index()
    # drop index and pass_email columns
    df = df.drop(columns={'index', 'pass_email'})
    return df

ticket_df = lookup_uuid(ticket_df, client)

[DEBUG][2023-12-23 19:19:00,593][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/abstract-flame-407818/jobs?prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 19:19:00,835][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/queries/466cffbf-7eb5-4d90-8a08-0bfe737a073e?maxResults=0&location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 19:19:01,016][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/queries/466cffbf-7eb5-4d90-8a08-0bfe737a073e?fields=jobReference%2CtotalRows%2CpageToken%2Crows&location=US&formatOptions.useInt64Timestamp=True&prettyPrint=false HTTP/1.1" 200 None
[INFO ][2023-12-23 19:19:01,018][68093012:0025] : query 32 rows from passengers table
[DEBUG][2023-12-23 19:19:01,019][68093012:0030] : preparing for join
[INFO ][2023-12-23 19:19:01,021][68093012:0034] : looked up UUID


Load tickets columns into a bigquery table

In [217]:
ticket_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['tickets']['table_name']}"
ticket_schema = schema=TABLE_METADATA['tickets']['schema']
load_table(ticket_df, client, ticket_table_name, ticket_schema)

[INFO ][2023-12-23 19:19:48,501][4006487939:0032] : loading table: 'abstract-flame-407818.air_travel.tickets'
[DEBUG][2023-12-23 19:19:49,348][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/abstract-flame-407818/jobs?uploadType=multipart HTTP/1.1" 200 2489
[DEBUG][2023-12-23 19:19:49,506][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/f66a059f-31bb-4c6d-9400-f9a0d3720653?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 19:19:49,507][retry:0233] : Retrying due to , sleeping 0.6s ...
[DEBUG][2023-12-23 19:19:50,225][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/abstract-flame-407818/jobs/f66a059f-31bb-4c6d-9400-f9a0d3720653?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2023-12-23 19:19:50,226][retry:0233] : Retrying due to , sleeping 0.4s ...
[DEBUG][2023-12-23 19:19:50,794][connectionpool:0546] : https://bi

Added primary keys to the tables

In [ ]:
ALTER TABLE `air_travel.airlines` ADD PRIMARY KEY (airline_iata) NOT ENFORCED;

ALTER TABLE `air_travel.airports` ADD PRIMARY KEY (airport_iata) NOT ENFORCED;

ALTER TABLE `air_travel.passengers` ADD PRIMARY KEY (UUID) NOT ENFORCED;

ALTER TABLE `air_travel.tickets` ADD PRIMARY KEY (eticket_num) NOT ENFORCED;